# metaflow_magicdir

> Save Entire Directories Into Metaflow's Metadata Store

## Install

`pip install metaflow_magicdir`

## How to use

You can use `@magicdir` to pass local directories between metaflow steps.  This will also work remotely.

In [ ]:
%%writefile examples/example_flow.py

from metaflow import FlowSpec, step
from metaflow_magicdir import magicdir


class MagicDirFlow(FlowSpec):

    @magicdir(dir='mydir')
    @step
    def start(self):
        with open('mydir/output1', 'w') as f:
            f.write('hello world')
        with open('mydir/output2', 'w') as f:
            f.write('hello world again')
        self.next(self.end)

    @magicdir(dir='mydir')
    @step
    def end(self):
        print('first', open('mydir/output1').read())
        print('second', open('mydir/output1').read())

if __name__ == "__main__":
    MagicDirFlow()

Overwriting examples/example_flow.py


If you run the above flow, you will see the following output:

In [ ]:
!python examples/example_flow.py run

Metaflow 2.5.4 executing MagicDirFlow for user:hamel
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-04-18 13:40:59.728 Workflow starting (run-id 1650314459725082):
2022-04-18 13:40:59.734 [1650314459725082/start/1 (pid 12408)] Task is starting.
2022-04-18 13:41:00.472 [1650314459725082/start/1 (pid 12408)] Task finished successfully.
2022-04-18 13:41:00.479 [1650314459725082/end/2 (pid 12412)] Task is starting.
2022-04-18 13:41:01.161 [1650314459725082/end/2 (pid 12412)] first hello world
2022-04-18 13:41:01.245 [1650314459725082/end/2 (pid 12412)] second hello world
2022-04-18 13:41:01.246 [1650314459725082/end/2 (pid 12412)] Task finished successfully.
2022-04-18 13:41:01.247 Done!


In [ ]:
#hide
from pathlib import Path
assert Path('mydir/output1').exists()
assert Path('mydir/output2').exists()

You can retrieve the results from the above Flow with the client api and `extract_magicdir`:

Let's first remove the directory if it exists:

In [ ]:
!rm -rf mydir/ #remove the directory if it exists

In [ ]:
from metaflow import Flow
from metaflow_magicdir import extract_magicdir
run_data = Flow('MagicDirFlow').latest_successful_run.data
extract_magicdir(run_data)

We can now inspect the contents of this directory to see it's contents!

In [ ]:
!ls mydir/

output1 output2


### `magicdir` with `foreach`

Nothing special is required to use `magicdir` with foreach.  Consider the following modification to the above flow:

In [ ]:
%%writefile examples/mapflow.py

from metaflow import FlowSpec, step
from metaflow_magicdir import magicdir


class MagicDirMapFlow(FlowSpec):
    """Show how magic directories work with foreach"""

    @step
    def start(self):
        self.step_num = range(5)
        self.next(self.write, foreach='step_num')

    @magicdir(dir='my_map_dir')
    @step
    def write(self):
        self.step_idx = self.input # metaflow gives self.input a value from `step_num` from the prior step
        with open(f'my_map_dir/{self.step_idx}.txt', 'w') as f:
            f.write(f'this is step {self.step_idx}')
        self.next(self.read)

    @magicdir(dir='my_map_dir')
    @step
    def read(self):
        print('file contents:', open(f'my_map_dir/{self.step_idx}.txt').read())
        self.next(self.join)
    
    @step
    def join(self, inputs):
        print(f"step numbers were: {[i.step_idx for i in inputs]}")
        self.next(self.end)

    @step
    def end(self): pass

if __name__ == "__main__":
    MagicDirMapFlow()

if __name__ == "__main__":
    MagicDirMapFlow()

Overwriting examples/mapflow.py


In [ ]:
!python examples/mapflow.py run

Metaflow 2.5.4 executing MagicDirMapFlow for user:hamel
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-04-18 13:41:56.687 Workflow starting (run-id 1650314516684584):
2022-04-18 13:41:56.695 [1650314516684584/start/1 (pid 12420)] Task is starting.
2022-04-18 13:41:57.444 [1650314516684584/start/1 (pid 12420)] Foreach yields 5 child steps.
2022-04-18 13:41:57.445 [1650314516684584/start/1 (pid 12420)] Task finished successfully.
2022-04-18 13:41:57.452 [1650314516684584/write/2 (pid 12423)] Task is starting.
2022-04-18 13:41:57.459 [1650314516684584/write/3 (pid 12424)] Task is starting.
2022-04-18 13:41:57.466 [1650314516684584/write/4 (pid 12425)] Task is starting.
2022-04-18 13:41:57.473 [1650314516684584/write/5 (pid 12426)] Task is starting.
2022-04-18 13:41:57.481 [1650314516684584/write/6 (pid 12427)] Task is starting.
2022-04-18 13:41:58.438 [1650314516684584/write/3 (pid 12424)] Task finished successfully.
2022-04-18 13:41:58.450 [

In [ ]:
#hide
_dir = Path('my_map_dir')
assert len(list(_dir.glob('*'))) == 5

In [ ]:
#hide
!rm -rf mydir my_map_dir